# HIGH-DIMENSIONAL METRICS IN R

## 2. How to get started

In [1]:
library(hdm)
library(stats)

Warning message:
"package 'hdm' was built under R version 4.2.1"


In [115]:
data(GrowthData)
dim(GrowthData)
## [1] 90 63
y = GrowthData[, 1, drop = F]
d = GrowthData[, 3, drop = F]
x = as.matrix(GrowthData)[, -c(1, 2, 3)]

# a = (rlasso(x = x, y = y)

[1] 90 63

In [118]:
rlasso(x = x, y = y)


Call:
rlasso.default(x = x, y = y)

Coefficients:
(Intercept)        bmp1l       freeop      freetar          h65         hm65  
    0.05810     -0.07557      0.00000      0.00000      0.00000      0.00000  
       hf65          p65         pm65         pf65          s65         sm65  
    0.00000      0.00000      0.00000      0.00000      0.00000      0.00000  
       sf65       fert65       mort65     lifee065        gpop1        fert1  
    0.00000      0.00000      0.00000      0.00000      0.00000      0.00000  
      mort1      invsh41      geetot1      geerec1         gde1       govwb1  
    0.00000      0.00000      0.00000      0.00000      0.00000      0.00000  
    govsh41     gvxdxe41       high65      highm65      highf65      highc65  
    0.00000      0.00000      0.00000      0.00000      0.00000      0.00000  
   highcm65     highcf65      human65     humanm65     humanf65        hyr65  
    0.00000      0.00000      0.00000      0.00000      0.00000      0.00000  
 

In [117]:
a = 5

## 4. Inference on Target Regression Coefficients

### 4.1. Intuition for the Orthogonality Principle in Linear Models via Partialling Out.

In [469]:
set.seed(1)
n = 5000
p = 20
X = matrix(rnorm(n * p), ncol = p)
colnames(X) = c("d", paste("x", 1:19, sep = ""))
xnames = colnames(X)[-1]
beta = rep(1, 20)
y = X %*% beta + rnorm(n)
dat = data.frame(y = y, X)
#save(dat, file = "../data/4_1.csv")
#write.csv(dat,"../data/4_1.csv", row.names = FALSE)


In [471]:
# full fit
fmla = as.formula(paste("y ~ ", paste(colnames(X), collapse = "+")))
full.fit = lm(fmla, data = dat)
summary(full.fit)$coef["d", 1:2]

Estimate Std. Error 
0.97807455 0.01371225

In [485]:
fmla.y = as.formula(paste("y ~ ", paste(xnames, collapse = "+")))
fmla.d = as.formula(paste("d ~ ", paste(xnames, collapse = "+")))
# partial fit via ols
rY = lm(fmla.y, data = dat)$res
rD = lm(fmla.d, data = dat)$res
partial.fit.ls = lm(rY ~ rD)
summary(partial.fit.ls)$coef["rD", 1:2]

Estimate Std. Error 
0.97807455 0.01368616

In [488]:
rY = rlasso(fmla.y, data = dat)$res
rD = rlasso(fmla.d, data = dat)$res
partial.fit.postlasso = lm(rY ~ rD)
summary(partial.fit.postlasso)$coef["rD", 1:2]

Estimate Std. Error 
0.97273870 0.01368677

### 4.2. Inference: Confidence Intervals and Significance Testing. The function rlassoEffects

In [496]:
set.seed(1)
n = 100 #sample size
p = 100 # number of variables
s = 3 # nubmer of non-zero variables
X = matrix(rnorm(n * p), ncol = p)
colnames(X) <- paste("X", 1:p, sep = "")
beta = c(rep(3, s), rep(0, p - s))
y = 1 + X %*% beta + rnorm(n)
data = data.frame(cbind(y, X))
#write.csv(data,"../data/4_2.csv", row.names = FALSE)
colnames(data)[1] <- "y"
fm = paste("y ~", paste(colnames(X), collapse = "+"))
fm = as.formula(fm)

In [497]:
# lasso.effect = rlassoEffects(X, y, index=c(1,2,3,50))
lasso.effect = rlassoEffects(fm, I = ~X1 + X2 + X3 + X50, data = data)
print(lasso.effect)


Call:
rlassoEffects.formula(formula = fm, data = data, I = ~X1 + X2 + 
    X3 + X50)

Coefficients:
     X1       X2       X3      X50  
2.94448  3.04127  2.97540  0.07196  



In [499]:
summary(lasso.effect)

[1] "Estimates and significance testing of the effect of target variables"
    Estimate. Std. Error t value Pr(>|t|)    
X1    2.94448    0.08815  33.404   <2e-16 ***
X2    3.04127    0.08389  36.253   <2e-16 ***
X3    2.97540    0.07804  38.127   <2e-16 ***
X50   0.07196    0.07765   0.927    0.354    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



In [500]:
confint(lasso.effect)


,2.5 %,97.5 %
X1,2.77171308,3.1172421
X2,2.87685121,3.2056979
X3,2.82244962,3.1283583
X50,-0.08022708,0.2241377


In [501]:
confint(lasso.effect, level = 0.95, joint = TRUE)


,2.5 %,97.5 %
X1,2.7279477,3.1610075
X2,2.8371214,3.2454278
X3,2.7833176,3.1674903
X50,-0.1154509,0.2593615


### 4.3. Application: the effect of gender on wage.

In [311]:
library(hdm)
data(cps2012)
X <- model.matrix(~-1 + female + female:(widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) + +(widowed +
divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so +
we + exp1 + exp2 + exp3)^2, data = cps2012)
dim(X)
X <- X[, which(apply(X, 2, var) != 0)] # exclude all constant variables
dim(X)
index.gender <- grep("female", colnames(X))
index.gender <- grep("female", colnames(X))
y <- cps2012$lnw

[1] 29217   136

[1] 29217   116

In [319]:
effects.female <- rlassoEffects(x = X, y = y, index = index.gender)
summary(effects.female)

[1] "Estimates and significance testing of the effect of target variables"
                    Estimate. Std. Error t value Pr(>|t|)    
female              -0.154923   0.050162  -3.088 0.002012 ** 
female:widowed       0.136095   0.090663   1.501 0.133325    
female:divorced      0.136939   0.022182   6.174 6.68e-10 ***
female:separated     0.023303   0.053212   0.438 0.661441    
female:nevermarried  0.186853   0.019942   9.370  < 2e-16 ***
female:hsd08         0.027810   0.120914   0.230 0.818092    
female:hsd911       -0.119335   0.051880  -2.300 0.021435 *  
female:hsg          -0.012890   0.019223  -0.671 0.502518    
female:cg            0.010139   0.018327   0.553 0.580114    
female:ad           -0.030464   0.021806  -1.397 0.162405    
female:mw           -0.001063   0.019192  -0.055 0.955811    
female:so           -0.008183   0.019357  -0.423 0.672468    
female:we           -0.004226   0.021168  -0.200 0.841760    
female:exp1          0.004935   0.007804   0.632 0.527139

In [320]:
joint.CI <- confint(effects.female, level = 0.95, joint = TRUE)
joint.CI

,2.5 %,97.5 %
female,-0.29392716,-0.01591940
female:widowed,-0.13309530,0.40528626
female:divorced,0.07492960,0.19894917
female:separated,-0.11641774,0.16302326
female:nevermarried,0.12938077,0.24432620
female:hsd08,-0.37364347,0.42926409
female:hsd911,-0.26870533,0.03003525
female:hsg,-0.06502795,0.03924839
female:cg,-0.04157113,0.06184824
female:ad,-0.09569738,0.03476989


### 4.4. Application: Estimation of the treatment effect in a linear model with many confounding factors

In [2]:
data(GrowthData)
dim(GrowthData)
## [1] 90 63
y = GrowthData[, 1, drop = F]
d = GrowthData[, 3, drop = F]
X = as.matrix(GrowthData)[, -c(1, 2, 3)]
varnames = colnames(GrowthData)

[1] 90 63

In [3]:
x1 = as.matrix(GrowthData)[, -c(1, 3)]

In [4]:
xnames = varnames[-c(1, 2, 3)] # names of X variables
dandxnames = varnames[-c(1, 2)] # names of D and X variables
# create formulas by pasting names (this saves typing times)
fmla = as.formula(paste("Outcome ~ ", paste(dandxnames, collapse = "+")))
ls.effect = lm(fmla, data = GrowthData)

In [350]:
stats:::qr.lm(object)

$qr
   (Intercept)      gdpsh465        bmp1l       freeop       freetar
1   -9.4868330 -73.076189875 -1.600871443 -2.088070386 -0.2688004527
2    0.1054093   8.454532979 -1.121146644  0.228693062 -0.1352109185
3    0.1054093  -0.153543593 -2.065490021  0.143035910 -0.0255253643
4    0.1054093   0.003745550  0.007092746 -0.652699019  0.0100417362
5    0.1054093   0.051397832 -0.038251710  0.130489097 -0.1532203197
6    0.1054093   0.044713582 -0.117878028  0.046966438 -0.0803360728
7    0.1054093  -0.030357993 -0.066044277 -0.070287481 -0.0602104846
8    0.1054093  -0.012652153  0.056129689  0.014449984  0.0021466978
9    0.1054093  -0.173459731  0.032761428 -0.182553732 -0.0177601769
10   0.1054093  -0.065641439  0.030164810 -0.161746212  0.0477454741
11   0.1054093   0.078942905 -0.127181190 -0.091831330 -0.1216248075
12   0.1054093   0.042481822 -0.012197156 -0.202671516 -0.1647674703
13   0.1054093  -0.061372703  0.164424578 -0.085767624  0.0078096455
14   0.1054093   0.038469121 -

In [340]:
correlation = FALSE; symbolic.cor = FALSE

z <- object
p <- z$rank
rdf <- z$df.residual
if (p == 0) {
    r <- z$residuals
    n <- length(r)
    w <- z$weights
    if (is.null(w)) {
        rss <- sum(r^2)
    } else {
        rss <- sum(w * r^2)
        r <- sqrt(w) * r
    }
    resvar <- rss/rdf
    ans <- z[c("call", "terms", if(!is.null(z$weights)) "weights")]
    class(ans) <- "summary.lm"
    ans$aliased <- is.na(coef(object))  # used in print method
    ans$residuals <- r
    ans$df <- c(0L, n, length(ans$aliased))
    ans$coefficients <- matrix(NA, 0L, 4L)
    dimnames(ans$coefficients) <-
        list(NULL, c("Estimate", "Std. Error", "t value", "Pr(>|t|)"))
    ans$sigma <- sqrt(resvar)
    ans$r.squared <- ans$adj.r.squared <- 0
    return(ans)
}
if (is.null(z$terms))
stop("invalid 'lm' object:  no 'terms' component")
if(!inherits(object, "lm"))
warning("calling summary.lm(<fake-lm-object>) ...")
Qr <- stats:::qr.lm(object)
n <- NROW(Qr$qr)
if(is.na(z$df.residual) || n - p != z$df.residual)
    warning("residual degrees of freedom in object suggest this is not an \"lm\" fit")
## do not want missing values substituted here
r <- z$residuals
f <- z$fitted.values
w <- z$weights
if (is.null(w)) {
    mss <- if (attr(z$terms, "intercept"))
        sum((f - mean(f))^2) else sum(f^2)
    rss <- sum(r^2)
} else {
    mss <- if (attr(z$terms, "intercept")) {
        m <- sum(w * f /sum(w))
        sum(w * (f - m)^2)
    } else sum(w * f^2)
    rss <- sum(w * r^2)
    r <- sqrt(w) * r
}
resvar <- rss/rdf
## see thread at https://stat.ethz.ch/pipermail/r-help/2014-March/367585.html
if (is.finite(resvar) &&
    resvar < (mean(f)^2 + var(f)) * 1e-30)  # a few times .Machine$double.eps^2
    warning("essentially perfect fit: summary may be unreliable")
p1 <- 1L:p
R <- chol2inv(Qr$qr[p1, p1, drop = FALSE])
se <- sqrt(diag(R) * resvar)

In [336]:
object = ls.effect
z <- object
p <- z$rank
rdf <- z$df.residual
r <- z$residuals

In [5]:
summary(ls.effect)$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),0.247160893,0.78450163,0.31505466,0.755056170
gdpsh465,-0.009377989,0.02988773,-0.31377391,0.756018518
bmp1l,-0.068862679,0.03253065,-2.11685513,0.043289718
freeop,0.080068974,0.20786400,0.38519885,0.703000838
freetar,-0.488962605,0.41816285,-1.16931143,0.252136477
h65,-2.362098638,0.85729167,-2.75530338,0.010192435
hm65,0.707143400,0.52314511,1.35171560,0.187285919
hf65,1.693448425,0.50318881,3.36543337,0.002232683
p65,0.265526695,0.16429407,1.61616729,0.117271229
pm65,0.136952626,0.15121749,0.90566657,0.372840111


In [6]:
dX = as.matrix(cbind(d, X))
lasso.effect = rlassoEffect(x = X, y = y, d = d, method = "partialling out")
summary(lasso.effect)

[1] "Estimates and significance testing of the effect of target variables"
     Estimate. Std. Error t value Pr(>|t|)    
[1,]  -0.04981    0.01394  -3.574 0.000351 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



In [7]:
doublesel.effect = rlassoEffect(x = X, y = y, d = d, method = "double selection")
summary(doublesel.effect)

[1] "Estimates and significance testing of the effect of target variables"
         Estimate. Std. Error t value Pr(>|t|)   
gdpsh465  -0.05001    0.01579  -3.167  0.00154 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



In [9]:
library(xtable)
table = rbind(summary(ls.effect)$coef["gdpsh465", 1:2], summary(lasso.effect)$coef[,
1:2], summary(doublesel.effect)$coef[, 1:2])
colnames(table) = c("Estimate", "Std. Error") #names(summary(full.fit)£coef)[1:2]
rownames(table) = c("full reg via ols", "partial reg
via post-lasso ", "partial reg via double selection")
tab = xtable(table, digits = c(2, 2, 5))
tab

,Estimate,Std. Error
,<dbl>,<dbl>
full reg via ols,-0.009377989,0.02988773
partial reg via post-lasso,-0.049811465,0.01393636
partial reg via double selection,-0.050005855,0.01579138


## 5. Instrumental Variable Estimation in a High-Dimensional Setting

### 5.2. Application: Economic Development and Institutions.

In [2]:
data(AJR)
y = AJR$GDP
d = AJR$Exprop
z = AJR$logMort
x = model.matrix(~-1 + (Latitude + Latitude2 + Africa + Asia + Namer + Samer)^2,
data = AJR)
dim(x)


[1] 64 21

In [3]:
AJR.Xselect = rlassoIV(GDP ~ Exprop + (Latitude + Latitude2 + Africa + Asia + Namer +
Samer)^2 | logMort + (Latitude + Latitude2 + Africa + Asia + Namer + Samer)^2,
data = AJR, select.X = TRUE, select.Z = FALSE)
summary(AJR.Xselect)


[1] "Estimation and significance testing of the effect of target variables in the IV regression model"
       coeff.    se. t-value p-value   
Exprop 0.8450 0.2699   3.131 0.00174 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1




In [4]:
confint(AJR.Xselect)

           2.5 %   97.5 %
Exprop 0.3159812 1.374072


In [5]:
# parialling out by linear model
fmla.y = GDP ~ (Latitude + Latitude2 + Africa + Asia + Namer + Samer)^2
fmla.d = Exprop ~ (Latitude + Latitude2 + Africa + Asia + Namer + Samer)^2
fmla.z = logMort ~ (Latitude + Latitude2 + Africa + Asia + Namer + Samer)^2
rY = lm(fmla.y, data = AJR)$res
rD = lm(fmla.d, data = AJR)$res
rZ = lm(fmla.z, data = AJR)$res
# ivfit.lm = tsls(y=rY,d=rD, x=NULL, z=rZ, intercept=FALSE)
ivfit.lm = tsls(rY ~ rD | rZ, intercept = FALSE)
print(cbind(ivfit.lm$coef, ivfit.lm$se), digits = 3)

   [,1] [,2]
rD 1.27 1.73


In [6]:
# parialling out by lasso
rY = rlasso(fmla.y, data = AJR)$res
rD = rlasso(fmla.d, data = AJR)$res
rZ = rlasso(fmla.z, data = AJR)$res
# ivfit.lasso = tsls(y=rY,d=rD, x=NULL, z=rZ, intercept=FALSE)
ivfit.lasso = tsls(rY ~ rD | rZ, intercept = FALSE)

In [10]:
summary(ivfit.lasso)

[1] "Estimates and Significance Testing from from tsls"
   Estimate Std. Error t value p value   
rD   0.8450     0.2699   3.131 0.00174 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1




### 5.3. Application: Impact of Eminent Domain Decisions on Economic Outcomes.

In [45]:
data(EminentDomain)
z <- as.matrix(EminentDomain$logGDP$z)
x <- as.matrix(EminentDomain$logGDP$x)
y <- EminentDomain$logGDP$y
d <- EminentDomain$logGDP$d
x <- x[, apply(x, 2, mean, na.rm = TRUE) > 0.05] #
z <- z[, apply(z, 2, mean, na.rm = TRUE) > 0.05] #

In [46]:
ED.ols = lm(y ~ cbind(d, x))
ED.2sls = tsls(y = y, d = d, x = x, z = z[, 1:2], intercept = FALSE)

In [47]:
lasso.IV.Z = rlassoIV(x = x, d = d, y = y, z = z, select.X = FALSE, select.Z = TRUE)
summary(lasso.IV.Z)

[1] "Estimates and significance testing of the effect of target variables in the IV regression model"
   coeff.    se. t-value p-value
d1 0.4146 0.2902   1.428   0.153




In [22]:
confint(lasso.IV.Z)

        2.5 %    97.5 %
d1 -0.1542764 0.9834796


In [49]:
lasso.IV.XZ = rlassoIV(x = x, d = d, y = y, z = z, select.X = TRUE, select.Z = TRUE)
summary(lasso.IV.XZ)

Estimates and Significance Testing of the effect of target variables in the IV regression model 
     coeff.      se. t-value p-value
d1 -0.02383  0.12851  -0.185   0.853




In [24]:
confint(lasso.IV.XZ)

        2.5 %    97.5 %
d1 -0.2757029 0.2280335


In [30]:
summary(ED.ols)$coef[2, 1:2]

Estimate  Std. Error 
0.007864732 0.009865927

In [31]:
cbind(ED.2sls$coef[1], ED.2sls$se[1])

d1,-0.01073327,0.03376636


In [32]:
library(xtable)
table = matrix(0, 4, 2)
table[1, ] = summary(ED.ols)$coef[2, 1:2]
table[2, ] = cbind(ED.2sls$coef[1], ED.2sls$se[1])
table[3, ] = summary(lasso.IV.Z)[, 1:2]

[1] "Estimates and significance testing of the effect of target variables in the IV regression model"
   coeff.    se. t-value p-value
d1 0.4146 0.2902   1.428   0.153




In [27]:
table

0.007864732,0.009865927
-0.010733269,0.033766362
0.414601641,0.290249208
0.000000000,0.000000000


## 7. The Lasso Methods for Discovery of Significant Causes amongst Many Potential Causes, with Many Controls

In [50]:
# library(hdm) library(stats)
set.seed(1)
n = 100
p1 = 20
p2 = 20
D = matrix(rnorm(n * p1), n, p1) # Causes
W = matrix(rnorm(n * p2), n, p2) # Controls
X = cbind(D, W) # Regressors
Y = D[, 1] * 5 + W[, 1] * 5 + rnorm(n) #Outcome

In [233]:
# data = data.frame(cbind(Y, X))
# write.csv(data,"../data/7_.csv", row.names = FALSE)

In [51]:
confint(rlassoEffects(X, Y, index = c(1:p1)), joint = TRUE)

,2.5 %,97.5 %
V1,4.5145877,5.21430498
V2,-0.3142909,0.30494650
V3,-0.3524109,0.18678880
V4,-0.2542430,0.28738914
V5,-0.2765802,0.27627177
V6,-0.3214676,0.29422684
V7,-0.2262507,0.30094168
V8,-0.0473541,0.47366372
V9,-0.1865636,0.39023520
V10,-0.2372356,0.26411185
